# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('/home/vyacheslav/Data Scientist/IV ЧЕТВЕРТЬ/II. Рекомендательные системы/2 lesson/data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('/home/vyacheslav/Data Scientist/IV ЧЕТВЕРТЬ/II. Рекомендательные системы/2 lesson/data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
n_items_before = data_train['item_id'].nunique()

filtered_items = item_features.loc[item_features['department'] == 'MISC. TRANS.', 'item_id'].tolist()

data_train = prefilter_items(data_train, filter_top_popular=5000,
                                exclude_top_popular=None, 
                                exclude_top_unpopular=None,
                                exclude_not_sold_weeks=None,
                                exclude_items=None,
                                exclude_cheaper_than = None,
                                exclude_more_expensive_than=None
                               )

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
model = MainRecommender(user_item_matrix, weighting=True, show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [8]:
result['own_recommends'] = result.apply(lambda row: model.get_own_recommends(user_id=row['user_id']), axis=1)

In [9]:
%time
result['similar_items_recommends'] = result.apply(lambda row: model.get_similar_items_recommendation(user_id=row['user_id']), axis=1)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 10.7 µs
Error: Рекоммендаций меньше, чем ожидается. user_id: 75
Error: Рекоммендаций меньше, чем ожидается. user_id: 650
Error: Рекоммендаций меньше, чем ожидается. user_id: 729
Error: Рекоммендаций меньше, чем ожидается. user_id: 954
Error: Рекоммендаций меньше, чем ожидается. user_id: 1105
Error: Рекоммендаций меньше, чем ожидается. user_id: 1539
Error: Рекоммендаций меньше, чем ожидается. user_id: 1565
Error: Рекоммендаций меньше, чем ожидается. user_id: 1636
Error: Рекоммендаций меньше, чем ожидается. user_id: 1893
Error: Рекоммендаций меньше, чем ожидается. user_id: 1984
Error: Рекоммендаций меньше, чем ожидается. user_id: 1987
Error: Рекоммендаций меньше, чем ожидается. user_id: 2120
Error: Рекоммендаций меньше, чем ожидается. user_id: 2259
Error: Рекоммендаций меньше, чем ожидается. user_id: 2355
Error: Рекоммендаций меньше, чем ожидается. user_id: 2364
Error: Рекоммендаций меньше, чем ожидается. user_id: 2373
Error: Ре

In [10]:
%time
result['similar_users_recommends'] = result.apply(lambda row: model.get_similar_users_recommendation(user_id=row['user_id']), axis=1)

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 14.1 µs
Error: Рекоммендаций меньше, чем ожидается. user_id: 3
Error: Рекоммендаций меньше, чем ожидается. user_id: 16
Error: Рекоммендаций меньше, чем ожидается. user_id: 31
Error: Рекоммендаций меньше, чем ожидается. user_id: 33
Error: Рекоммендаций меньше, чем ожидается. user_id: 39
Error: Рекоммендаций меньше, чем ожидается. user_id: 63
Error: Рекоммендаций меньше, чем ожидается. user_id: 68
Error: Рекоммендаций меньше, чем ожидается. user_id: 73
Error: Рекоммендаций меньше, чем ожидается. user_id: 75
Error: Рекоммендаций меньше, чем ожидается. user_id: 83
Error: Рекоммендаций меньше, чем ожидается. user_id: 103
Error: Рекоммендаций меньше, чем ожидается. user_id: 108
Error: Рекоммендаций меньше, чем ожидается. user_id: 110
Error: Рекоммендаций меньше, чем ожидается. user_id: 124
Error: Рекоммендаций меньше, чем ожидается. user_id: 138
Error: Рекоммендаций меньше, чем ожидается. user_id: 139
Error: Рекоммендаций меньше, чем ож

Error: Рекоммендаций меньше, чем ожидается. user_id: 1581
Error: Рекоммендаций меньше, чем ожидается. user_id: 1598
Error: Рекоммендаций меньше, чем ожидается. user_id: 1612
Error: Рекоммендаций меньше, чем ожидается. user_id: 1623
Error: Рекоммендаций меньше, чем ожидается. user_id: 1631
Error: Рекоммендаций меньше, чем ожидается. user_id: 1638
Error: Рекоммендаций меньше, чем ожидается. user_id: 1639
Error: Рекоммендаций меньше, чем ожидается. user_id: 1651
Error: Рекоммендаций меньше, чем ожидается. user_id: 1664
Error: Рекоммендаций меньше, чем ожидается. user_id: 1676
Error: Рекоммендаций меньше, чем ожидается. user_id: 1677
Error: Рекоммендаций меньше, чем ожидается. user_id: 1681
Error: Рекоммендаций меньше, чем ожидается. user_id: 1693
Error: Рекоммендаций меньше, чем ожидается. user_id: 1710
Error: Рекоммендаций меньше, чем ожидается. user_id: 1740
Error: Рекоммендаций меньше, чем ожидается. user_id: 1745
Error: Рекоммендаций меньше, чем ожидается. user_id: 1767
Error: Рекомме

-----

In [11]:
result.head(2)

,user_id,actual,own_recommends,similar_items_recommends,similar_users_recommends
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9527290, 1043064, 1082185, 1075074, 10149640]","[9526410, 845262, 999999, 862349, 13877192]","[9527290, 1056509, 1070820, 1098066, 1024306]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1092026, 1053690, 9527494, 1092937, 9337581]","[1076875, 1092026, 9527160, 1000736, 9337369]","[1092026, 8090521, 1110764, 1118393]"


In [12]:
result.apply(lambda row: precision_at_k(row['own_recommends'], row['actual']), axis=1).mean()

/home/vyacheslav/Data Scientist/IV ЧЕТВЕРТЬ/II. Рекомендательные системы/4 lesson/src/metrics.py:24: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


0.2881514257620452

In [13]:
result.apply(lambda row: precision_at_k(row['similar_items_recommends'], row['actual']), axis=1).mean()

0.06725663716814159

In [14]:
result.head(20).apply(lambda row: precision_at_k(row['similar_users_recommends'], row['actual']), axis=1).mean()

0.15